### 스키니랩 다이어트보조제 

In [27]:
from selenium import webdriver
import time
import pandas as pd 
import pymongo as mg 

In [81]:
browser = webdriver.Chrome(executable_path='../chromedriver.exe')

In [82]:
browser.get('https://www.skinnylab.co.kr/shop/diet/cissus?JTdCJTIyY29uZGl0aW9ucyUyMiUzQSU3QiUyMmNhdGVnb3J5Q2xzZkJTZXElMjIlM0ElMjI2JTIyJTJDJTIyc2VhcmNoV29yZCUyMiUzQSUyMiUyMiUyQyUyMm1pblNhbGVQcmljZSUyMiUzQTE5NTAwJTJDJTIybWF4U2FsZVByaWNlJTIyJTNBMjM2NTAwJTdEJTJDJTIycGFnZSUyMiUzQTElMkMlMjJzaXplJTIyJTNBMTAlMkMlMjJzb3J0JTIyJTNBJTIydmlld0NvdW50JTJDREVTQyUyMiU3RA==')

### 글 가져오기
- 상품 클릭 tag: #goods > div.container-wrap > div > ul > li:nth-child(1)
- 상품명 tag: #goodsDetail > div.content > div.head > h2 > span.main
- 상품리뷰 클릭 tag: #content-main > div.fixed-tab-menu > div > div > ul > li:nth-child(3)
- 리뷰 번들 tag: #review > div.review-accordion > dl:nth-child({})
- 리뷰 작성자 tag: .name
- 리뷰 별점 tag: .fill  / #review > div.review-accordion > dl:nth-child(1) > dt > div > p.grade-star.small
- 리뷰 날짜 tag: .date
- 리뷰 내용 tag: .review-contents-detail

### 페이지네이션
- next page : #review > div.common-pagination > div > a.paging-btn.next

### 컬럼 
product_name, review_name, review_title, review_content, review_date, review_rating

In [57]:
#페이지당 상품 묶음 개수 체크        
post_bundle = browser.find_elements_by_css_selector('.main')
len(post_bundle)

10

In [128]:
#상품 이름 및 클릭 
browser.find_element_by_css_selector('#goods > div.container-wrap > div > ul > li:nth-child({})'.format(1)).click()
product_name = browser.find_element_by_css_selector('#goodsDetail > div.content > div.head > h2 > span.main').text
product_name

'스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분'

In [134]:
#상품리뷰 클릭 
browser.find_element_by_css_selector('#content-main > div.fixed-tab-menu > div > div > ul > li:nth-child(3)').click()

In [135]:
#리뷰 번들 
review_bundle = browser.find_element_by_css_selector('#review > div.review-accordion > dl:nth-child({})'.format(10))

In [138]:
#글쓴이, 내용, 날짜, 별점 
review_name = review_bundle.find_element_by_css_selector('.name').text
review_content = review_bundle.find_element_by_css_selector('.review-contents-detail').text
review_date = review_bundle.find_element_by_css_selector('.date').text
review_rating = review_bundle.find_element_by_css_selector('.fill').get_attribute('aria-label')

In [139]:
[product_name, review_name, review_content, review_date, review_rating]

['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분',
 '이*정',
 '거의6**째먹고잇어요 맛은먹을만한데가격이너,무비싸네요 ㅜ ***.분해해준다길래먹고잇어요 가격이너무부담되네요',
 '2021.12.14',
 None]

In [ ]:
import time

client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_supplements_data_analysis']
collection = database['skinnylab_reviews']

product_columns_name = ['product_name', 'review_name', 'review_content', 'review_date', 'review_rating']
review_list = []

for product in range(1, 10):
    product_name = browser.find_element_by_css_selector('#goods > div.container-wrap > div > ul > li:nth-child({}) > div > a > figure > figcaption > p.title > span'.format(product)).text
    browser.find_element_by_css_selector('#goods > div.container-wrap > div > ul > li:nth-child({})'.format(product)).click()
    
    time.sleep(1)
    
    # 리뷰 클릭
    browser.find_element_by_css_selector('#content-main > div.fixed-tab-menu > div > div > ul > li:nth-child(3)').click()
    time.sleep(1)
    try:
        for next_page in range(1, 100):
            try:
                for number in range(1, 11):
                    try:
                        review_bundle = browser.find_element_by_css_selector('#review > div.review-accordion > dl:nth-child({})'.format(number))
                        
                        try:
                            review_name = review_bundle.find_element_by_css_selector('.name').text
                        except:
                            review_name = ""
                        
                        try:
                            review_content = review_bundle.find_element_by_css_selector('.review-contents-detail').text
                        except:
                            review_content = ""
                        
                        try:
                            review_date = review_bundle.find_element_by_css_selector('.date').text
                        except:
                            review_date = ""
                          
                        try: 
                            review_rating = ""  
                        except:
                            review_rating = ""
                            
                        single_review_list = [product_name, review_name, review_content, review_date, review_rating]
                        print(single_review_list)
                        time.sleep(1)
                        review_list.append(single_review_list)
                        df_review = pd.DataFrame(data= review_list, columns=product_columns_name)
                        data_dict = df_review.to_dict(orient='records')
                        collection.insert_many(data_dict)
                        browser.execute_script("window.scrollTo(0, 150)")
                        time.sleep(1)
                    except Exception as e:
                        break
                        print(f"Error occurred while scraping review data: {e}")
                # 페이지네이션을 찾아서 클릭
                next_button = browser.find_element_by_xpath('//*[@class="paging-btn next"]')
                next_button.click()
                time.sleep(1)
            
            except Exception as e:
                break
                print(f"Error occurred while navigating to the next page: {e}")
            time.sleep(1)
    except Exception as e:
        pass
        print(f"Error occurred while scraping product data: {e}")
    finally:
        browser.get('https://www.skinnylab.co.kr/shop/diet/cissus?JTdCJTIyY29uZGl0aW9ucyUyMiUzQSU3QiUyMmNhdGVnb3J5Q2xzZkJTZXElMjIlM0ElMjI2JTIyJTJDJTIyc2VhcmNoV29yZCUyMiUzQSUyMiUyMiUyQyUyMm1pblNhbGVQcmljZSUyMiUzQTE5NTAwJTJDJTIybWF4U2FsZVByaWNlJTIyJTNBMjM2NTAwJTdEJTJDJTIycGFnZSUyMiUzQTElMkMlMjJzaXplJTIyJTNBMTAlMkMlMjJzb3J0JTIyJTNBJTIydmlld0NvdW50JTJDREVTQyUyMiU3RA==')
        time.sleep(5)
browser.quit()

['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '가*****************♡', '노력만하시면. 구입은. 후회안합니다.  다만. 먹다가. 안먹으면.  다시. 그대로. 한순간변해요.  4년동한먹었던 나.   6**먹다가. 2달끊었는데.  다시. 복궈가 되네요.  참고하시고.  운동 하시면서.  먹는다면. 추천.      밥한끼안먹고. 이걸로. 2스틱. 500m. 물량을. 섭취한다면.  추천드립니다.  까놓고애기하니. 이글을. 관리자. 님들이 보신다면 좋아하지는않겠죠.   꾸준히 게속 드셔야합니다.   안그럼 비추천드려요.  😀   ^^', '2022.12.28', '']
['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '강*림', '먹기편하고 맛도 좋아요. 3주차라 **는 좀더 봐야할거 같아요.', '2022.10.09', '']
['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '정*', '아직까지 드라마틱한 **는 없지만 더 찌지 않았어요 \n맛있어서 먹는데는 부담없어요. \n여섯달까지 쭉 먹어보렵니다 ', '2022.09.05', '']
['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '임*하', '*국 추천받아서 ** 감량을 위해 구입했습니다. 부담스럽지 않은 맛에 아침저녁 먹기 전 1회씩 총 2회를 하루에 먹고 있는데요. 뭔가 섭취 후에 배출이 잘되는 느낌입니다~ 꾸준히 먹어보려구요. ', '2022.08.01', '']
['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '김*희', '시서스를 꾸준히 먹고 있는데\n매번 선물로 받다가 이번엔 제가 구매를 해야해서 스키랩으로 들어가 회원까지 가입하고 시서스를 구매했내요\n비오틴 사은품과 풋사과 잴리\n먹어보고 싶었던것을 시서스 주문으로 함께ㅡ받아보니 더없이 좋네요\n담엔 락토페린도 그매할 예정입니다', '2022.04.13', '']
['스키니랩 가벼워지는 시서스 다이어트 14포 x8박스 8주분', '심*수', 